# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [58]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

#sklearn package
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

#nlp pacakges
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [52]:
# load data from database
engine = create_engine('sqlite:///Udacity_Disaster_Response.db')
df = pd.read_sql_table("message_categories",con=engine)
#id is useless for prediction,
#original is the same as message, but in different language
#for now only use message
#we may add genre later
X = df[df.columns[1]]
Y = df[df.columns[4:]]

In [3]:
#some check
df.head()

id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  related  \
0  Un front froid se retrouve sur Cuba ce matin. ...  direct        1   
1                 Cyclone nan fini osinon li pa fini  direct        1   
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct        1   
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct        1   
4  facade ouest d Haiti et le reste du pays aujou...  direct        1   

   request  offer  aid_related  medical_help  medical_products      ...        \
0        0      0            0             0                 0      ...         
1        0      0            1             0                 0      ...         
2        0      0            0             0                 0      ...         
3        1      0            1             0                 1      ...         
4        0      0            0             0                 0      ...         

   aid_centers  other_infrastructure  weather_related  floods  storm  fire  \
0            0                     0                0       0      0     0   
1            0                     0                1       0      1     0   
2            0                     0                0       0      0     0   
3            0                     0                0       0      0     0   
4            0                     0                0       0      0     0   

   earthquake  cold  other_weather  direct_report  
0           0     0              0              0  
1           0     0              0              0  
2           0     0              0              0  
3           0     0              0              0  
4           0     0              0              0  

[5 rows x 40 columns]

In [45]:
df.shape

(26216, 40)

In [72]:
#2 for related 
df['related'].value_counts()

1    19906
0     6122
2      188
Name: related, dtype: int64

In [36]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [21]:
Y.head()

related  request  offer  aid_related  medical_help  medical_products  \
0        1        0      0            0             0                 0   
1        1        0      0            1             0                 0   
2        1        0      0            0             0                 0   
3        1        1      0            1             0                 1   
4        1        0      0            0             0                 0   

   search_and_rescue  security  military  child_alone      ...        \
0                  0         0         0            0      ...         
1                  0         0         0            0      ...         
2                  0         0         0            0      ...         
3                  0         0         0            0      ...         
4                  0         0         0            0      ...         

   aid_centers  other_infrastructure  weather_related  floods  storm  fire  \
0            0                     0                0       0      0     0   
1            0                     0                1       0      1     0   
2            0                     0                0       0      0     0   
3            0                     0                0       0      0     0   
4            0                     0                0       0      0     0   

   earthquake  cold  other_weather  direct_report  
0           0     0              0              0  
1           0     0              0              0  
2           0     0              0              0  
3           0     0              0              0  
4           0     0              0              0  

[5 rows x 36 columns]

In [28]:
#one message may have several categories
Y.sum(axis=1).head()

0    1
1    5
2    1
3    8
4    1
dtype: int64

In [44]:
#wee drop column child_alone since all value is zero 
#we also need to be very careful about the large
#sum value or small sum value, this means the data is
#is not evenly weighted
Y.sum()

related                   20282
request                    4474
offer                       118
aid_related               10860
medical_help               2084
medical_products           1313
search_and_rescue           724
security                    471
military                    860
child_alone                   0
water                      1672
food                       2923
shelter                    2314
clothing                    405
money                       604
missing_people              298
refugees                    875
death                      1194
other_aid                  3446
infrastructure_related     1705
transport                  1201
buildings                  1333
electricity                 532
tools                       159
hospitals                   283
shops                       120
aid_centers                 309
other_infrastructure       1151
weather_related            7297
floods                     2155
storm                      2443
fire    

In [53]:
Y=Y.drop(['child_alone'],axis=1)

In [54]:
Y.head()

related  request  offer  aid_related  medical_help  medical_products  \
0        1        0      0            0             0                 0   
1        1        0      0            1             0                 0   
2        1        0      0            0             0                 0   
3        1        1      0            1             0                 1   
4        1        0      0            0             0                 0   

   search_and_rescue  security  military  water      ...        aid_centers  \
0                  0         0         0      0      ...                  0   
1                  0         0         0      0      ...                  0   
2                  0         0         0      0      ...                  0   
3                  0         0         0      0      ...                  0   
4                  0         0         0      0      ...                  0   

   other_infrastructure  weather_related  floods  storm  fire  earthquake  \
0                     0                0       0      0     0           0   
1                     0                1       0      1     0           0   
2                     0                0       0      0     0           0   
3                     0                0       0      0     0           0   
4                     0                0       0      0     0           0   

   cold  other_weather  direct_report  
0     0              0              0  
1     0              0              0  
2     0              0              0  
3     0              0              0  
4     0              0              0  

[5 rows x 35 columns]

In [14]:
#make some detail check of the message and original
#they are same but different language
for i in range(8):
    print(str(i))
    print("message: "+df['message'][i])
    print("original: "+df['original'][i])

0
message: Weather update - a cold front from Cuba that could pass over Haiti
original: Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de pluie isolee sont encore prevues sur notre region ce soi
1
message: Is the Hurricane over or is it not over
original: Cyclone nan fini osinon li pa fini
2
message: Looking for someone but no name
original: Patnm, di Maryani relem pou li banm nouvel li ak timoun yo. Mesi se john jean depi Monben kwochi.
3
message: UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
original: UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
4
message: says: west side of Haiti, rest of the country today and tonight
original: facade ouest d Haiti et le reste du pays aujourd hui et ce soir
5
message: Information about the National Palace-
original: Informtion au nivaux palais nationl
6
message: Storm at sacred heart of jesus
o

In [17]:
df['genre'].value_counts(dropna=False)

news      13054
direct    10766
social     2396
Name: genre, dtype: int64

### 2. Write a tokenization function to process your text data

In [30]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

In [31]:
def tokenize(text):
    #normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]"," ",text.lower())
    #tokenize text
    tokens = word_tokenize(text)
    
    #lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return tokens

In [33]:
#make some test
# works
tokenize(X['message'][1])

['hurricane']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [42]:
pipeline = Pipeline([('vect',CountVectorizer(tokenizer=tokenize)),
                    ('tfidf',TfidfTransformer()),
                    ('clf',MultiOutputClassifier(estimator=LogisticRegression()))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [55]:
#split train and test
X_train,X_test,y_train,y_test = train_test_split(X,Y,random_state=42)

In [56]:
#fit the model
pipeline.fit(X_train,y_train);

C:\Users\zzguk\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\zzguk\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...e=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [57]:
y_predict = pipeline.predict(X_test)

In [71]:
for i in range(y_test.shape[1]):
    print("+++++++++++++   "+y_test.columns[i]+"   +++++++++++++++++")
    print(classification_report(y_test[y_test.columns[i]],y_predict[:,i]))
    #print(classification_report)

+++++++++++++   related   +++++++++++++++++
              precision    recall  f1-score   support

           0       0.73      0.41      0.52      1563
           1       0.83      0.95      0.89      4944
           2       0.00      0.00      0.00        47

   micro avg       0.82      0.82      0.82      6554
   macro avg       0.52      0.45      0.47      6554
weighted avg       0.80      0.82      0.79      6554

+++++++++++++   request   +++++++++++++++++
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5443
           1       0.83      0.52      0.64      1111

   micro avg       0.90      0.90      0.90      6554
   macro avg       0.87      0.75      0.79      6554
weighted avg       0.90      0.90      0.89      6554

+++++++++++++   offer   +++++++++++++++++
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6473
           1       1.00      0.01      0.02        81

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.99      0.51      0.51      6554
weighted avg       0.99      0.99      0.98      6554

+++++++++++++   other_infrastructure   +++++++++++++++++
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6271
           1       0.25      0.00      0.01       283

   micro avg       0.96      0.96      0.96      6554
   macro avg       0.60      0.50      0.49      6554
weighted avg       0.93      0.96      0.94      6554

+++++++++++++   weather_related   +++++++++++++++++
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      4781
           1       0.85      0.65      0.73      1773

   micro avg       0.87      0.87      0.87      6554
   macro avg       

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.